### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Italian
Lang Id = 7
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,che sono una cosa che non lo,206,208,che sono una cosa che non lo sa più,iBdSB7CjB0A,https://www.youtube.com/watch?v=iBdSB7CjB0A&t=...
1,è una cosa che non si,3373,3375,è una cosa che non si vede,0fjmn2TpzH4,https://www.youtube.com/watch?v=0fjmn2TpzH4&t=...
2,che non è una cosa da,322,323,che non è una cosa da poco,X9Z9iwK4whQ,https://www.youtube.com/watch?v=X9Z9iwK4whQ&t=...
3,con che cosa con una,80,83,con che cosa con una frusta,ZsnF4gwcYaI,https://www.youtube.com/watch?v=ZsnF4gwcYaI&t=80s
4,e non è una cosa da,210,212,e non è una cosa da poco,Efb1ZbVxHik,https://www.youtube.com/watch?v=Efb1ZbVxHik&t=...
...,...,...,...,...,...,...
195,non con,1471,1483,non con,oACEKF5pFcI,https://www.youtube.com/watch?v=oACEKF5pFcI&t=...
196,di cosa,108,110,di cosa,C2ZBTklACY4,https://www.youtube.com/watch?v=C2ZBTklACY4&t=...
197,sono le,241,256,sono le,nvuXhZ_2D38,https://www.youtube.com/watch?v=nvuXhZ_2D38&t=...
198,sono le,275,279,sono le,_OzrWRbAQtQ,https://www.youtube.com/watch?v=_OzrWRbAQtQ&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,vedere meglio e siamo qui allora quello che no...,1661,1669,vedere meglio e siamo qui allora quello che no...,GH1Zq1GarM4,https://www.youtube.com/watch?v=GH1Zq1GarM4&t=...
1,come tu hai fatto sì io ho fatto questa cosa h...,1372,1387,come tu hai fatto sì io ho fatto questa cosa h...,_9o21e4PwZs,https://www.youtube.com/watch?v=_9o21e4PwZs&t=...
2,non è quello ok adesso sto a casa di qualcuno ...,36,41,probabilmente non è quello ok adesso sto a cas...,9SesLBahFjc,https://www.youtube.com/watch?v=9SesLBahFjc&t=36s
3,quello che siamo perché lo siamo già stato pri...,20,25,a capire quello che siamo perché lo siamo già ...,L2CyvarDzi8,https://www.youtube.com/watch?v=L2CyvarDzi8&t=20s
4,che tutti noi prima o poi o abbiamo fatto o ab...,107,114,have che tutti noi prima o poi o abbiamo fatto...,sHTcserYi68,https://www.youtube.com/watch?v=sHTcserYi68&t=...
...,...,...,...,...,...,...
195,va beh per questa volta e solo perché mi sei,101,104,va beh per questa volta e solo perché mi sei s...,OhHQpWugTgU,https://www.youtube.com/watch?v=OhHQpWugTgU&t=...
196,a casa sì ci vuoi fare per qualche giorno da,342,349,soli a casa sì ci vuoi fare per qualche giorno...,9NfIfTul6tA,https://www.youtube.com/watch?v=9NfIfTul6tA&t=...
197,di tutti quindi non so mai a chi fa più bene,72,76,di tutti quindi non so mai a chi fa più bene,PgeHdfKIvwM,https://www.youtube.com/watch?v=PgeHdfKIvwM&t=72s
198,parlare con qualcuno ecco in questo tempo ho,2934,2938,parlare con qualcuno ecco in questo tempo ho p...,g5Vdi60pMfU,https://www.youtube.com/watch?v=g5Vdi60pMfU&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,tu hai sentito parlare di questa cosa semplice...,1469,1475,tu hai sentito parlare di questa cosa semplice...,aFjEiPXtL6w,https://www.youtube.com/watch?v=aFjEiPXtL6w&t=...
1,grazie per essere stato o per essere stata qui...,455,463,grazie per essere stato o per essere stata qui...,eljZW_3E79g,https://www.youtube.com/watch?v=eljZW_3E79g&t=...
2,perché questo video è stato molto più difficil...,58,64,perché questo video è stato molto più difficil...,brR9KopWpt4,https://www.youtube.com/watch?v=brR9KopWpt4&t=58s
3,grazie per essere stato o per essere stata qui...,494,501,grazie per essere stato o per essere stata qui...,3ArISj4cyPo,https://www.youtube.com/watch?v=3ArISj4cyPo&t=...
4,una domanda semplice questa è la nostra ultima...,637,647,buonasera corrado una domanda semplice questa ...,HAqfb5O8LaY,https://www.youtube.com/watch?v=HAqfb5O8LaY&t=...
...,...,...,...,...,...,...
195,chi lo sa però io mi sono sempre trovato bene ...,661,666,chi lo sa però io mi sono sempre trovato bene ...,ECz_Z0WKrmE,https://www.youtube.com/watch?v=ECz_Z0WKrmE&t=...
196,dietro quindi noi dobbiamo lavorare qui dentro...,406,413,dietro quindi noi dobbiamo lavorare qui dentro...,vKHfReL_erE,https://www.youtube.com/watch?v=vKHfReL_erE&t=...
197,tutto vediamo da chi viene quindi abbiamo capi...,2727,2733,tutto vediamo da chi viene quindi abbiamo capi...,OIGH0hnZJTk,https://www.youtube.com/watch?v=OIGH0hnZJTk&t=...
198,fare secondo te cosa stanno facendo insieme in...,319,325,fare secondo te cosa stanno facendo insieme in...,g3Y6sDokvlY,https://www.youtube.com/watch?v=g3Y6sDokvlY&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ciao come stai vuol dire che del tuo umore imp...,17,38,ciao come stai vuol dire che del tuo umore imp...,8PQ71A2qPRk,https://www.youtube.com/watch?v=8PQ71A2qPRk&t=17s
1,comunque ti ringrazio per essere stato in mia ...,800,812,comunque ti ringrazio per essere stato in mia ...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
2,andati a letto i bambini dopo un ora stavano t...,250,261,andati a letto i bambini dopo un ora stavano t...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,per cui in qualche modo possiamo vedere se riu...,820,830,per cui in qualche modo possiamo vedere se riu...,Ek0vTrfaxX0,https://www.youtube.com/watch?v=Ek0vTrfaxX0&t=...
4,seguendo però sempre la sensazione che si ha n...,739,748,seguendo però sempre la sensazione che si ha n...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
...,...,...,...,...,...,...
195,due conti io li ho fatti magari li avete fatti...,225,232,due conti io li ho fatti magari li avete fatti...,ANSaBS5_WZY,https://www.youtube.com/watch?v=ANSaBS5_WZY&t=...
196,fare qualcosa per proprio fare bene fare del b...,313,319,fare qualcosa per proprio fare bene fare del b...,el7DWhUsHxU,https://www.youtube.com/watch?v=el7DWhUsHxU&t=...
197,si sta facendo mettere da parte le patate dolc...,112,120,abbie si sta facendo mettere da parte le patat...,GElRvtF0wx0,https://www.youtube.com/watch?v=GElRvtF0wx0&t=...
198,quindi è molto importante essere sempre presen...,446,451,quindi è molto importante essere sempre presen...,EepIuTx7704,https://www.youtube.com/watch?v=EepIuTx7704&t=...


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,quindi venivo qui due volte a settimana il sab...,294,305,quindi venivo qui due volte a settimana il sab...,miyE4qxnx70,https://www.youtube.com/watch?v=miyE4qxnx70&t=...
1,quando mi è stato chiesto di partecipare a que...,5,13,quando mi è stato chiesto di partecipare a que...,WwxadFn4iV8,https://www.youtube.com/watch?v=WwxadFn4iV8&t=5s
2,sarà necessaria una preparazione quasi formale...,223,241,sarà necessaria una preparazione quasi formale...,ldpgAhQtANs,https://www.youtube.com/watch?v=ldpgAhQtANs&t=...
3,finale molto essenziale molto semplice ma cari...,1018,1030,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
4,la cosa interessante che si può vedere dalle f...,204,215,la cosa interessante che si può vedere dalle f...,yqH9YdpZihk,https://www.youtube.com/watch?v=yqH9YdpZihk&t=...
...,...,...,...,...,...,...
195,chi siete voi cosa volete da noi quindi era un...,1120,1126,chi siete voi cosa volete da noi quindi era un...,UZAJp5RxPUQ,https://www.youtube.com/watch?v=UZAJp5RxPUQ&t=...
196,in primo luogo ringrazio il nostro staff per e...,39,45,in primo luogo ringrazio il nostro staff per e...,oN74JtEBRmo,https://www.youtube.com/watch?v=oN74JtEBRmo&t=39s
197,a questo punto posso andare a chiudere possiam...,348,357,a questo punto posso andare a chiudere possiam...,1rX8sxTw4kc,https://www.youtube.com/watch?v=1rX8sxTw4kc&t=...
198,grande questo dipinto si chiama omaggio ad ele...,704,716,grande questo dipinto si chiama omaggio ad ele...,oAbwzLE2ecY,https://www.youtube.com/watch?v=oAbwzLE2ecY&t=...


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
1,finale molto essenziale molto semplice ma cari...,1018,1031,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
2,che stanno proprio nel nostro quartiere verame...,407,420,che stanno proprio nel nostro quartiere verame...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,ciao cari amici benvenuti nella mia cucina ogg...,0,9,ciao cari amici benvenuti nella mia cucina ogg...,AkW2coApj00,https://www.youtube.com/watch?v=AkW2coApj00&t=0s
4,quello che ci chiedete forse dal primo giorno ...,36,47,quello che ci chiedete forse dal primo giorno ...,9HuoF2bl2us,https://www.youtube.com/watch?v=9HuoF2bl2us&t=36s
...,...,...,...,...,...,...
195,facciamo alcuni esempi al singolare il pesce l...,82,98,facciamo alcuni esempi al singolare il pesce l...,8h0a_160G5Y,https://www.youtube.com/watch?v=8h0a_160G5Y&t=82s
196,in realtà non ho mai fatto il meccanico in vit...,277,289,in realtà non ho mai fatto il meccanico in vit...,PYxMGK97yrI,https://www.youtube.com/watch?v=PYxMGK97yrI&t=...
197,se anche tu vuoi cominciare a pubblicare video...,2,8,se anche tu vuoi cominciare a pubblicare video...,vJsbqEPp12o,https://www.youtube.com/watch?v=vJsbqEPp12o&t=2s
198,si ricevono zolfo bisogna decidere subito se t...,1511,1519,qualvolta si ricevono zolfo bisogna decidere s...,Z1rrA3lGDK8,https://www.youtube.com/watch?v=Z1rrA3lGDK8&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,https://www.youtube.com/watch?v=2HVKJg__wYk&t=...
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,https://www.youtube.com/watch?v=fomcMM1XA5A&t=...
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,https://www.youtube.com/watch?v=dMNIjzeZA1c&t=...
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U,https://www.youtube.com/watch?v=AwdMIo8Lv5U&t=42s
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
...,...,...,...,...,...,...
195,questa è in alto togliamo questi fini qui ne l...,579,602,questa è in alto togliamo questi fini qui ne l...,YRJ2CGwVuBQ,https://www.youtube.com/watch?v=YRJ2CGwVuBQ&t=...
196,una volta laureati infatti bisogna tuffarsi im...,227,236,una volta laureati infatti bisogna tuffarsi im...,G7a1frpwjc8,https://www.youtube.com/watch?v=G7a1frpwjc8&t=...
197,passiamo quindi ai dettagli finali con un penn...,988,997,passiamo quindi ai dettagli finali con un penn...,4iVtQnjgTvA,https://www.youtube.com/watch?v=4iVtQnjgTvA&t=...
198,va bene ok calmati dominick perchè alla fine q...,368,376,va bene ok calmati dominick perchè alla fine q...,SApMQKc1qPs,https://www.youtube.com/watch?v=SApMQKc1qPs&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [16]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [17]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [18]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [19]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [20]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [21]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [22]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [23]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [24]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [25]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [26]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [27]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [28]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [29]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [30]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [31]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [32]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [33]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [34]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [35]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [36]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [37]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [38]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [39]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [40]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [41]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [42]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [43]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [44]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [45]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [46]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1671273914
  nanos: 830650000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass